In [45]:
import sys
from transformers import AutoTokenizer

# Caution: path[0] is reserved for script path
sys.path.insert(1, '/home/arya/my_python/my_torch/cq')
from useful_function import truncate_string,fit_in_shape

In [39]:
model_name = "Salesforce/codet5p-770m"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def read_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

# Read and prepare data
codes = read_file('/home/arya/my_python/my_torch/cq/codesum_data/clean_train.code')
comments = read_file('/home/arya/my_python/my_torch/cq/codesum_data/clean_train.comment')


In [ ]:
# truncate all comments to 512 tokens
comments = [truncate_string(comment, 127, tokenizer) for comment in comments]

In [41]:
answers= read_file('/home/arya/my_python/my_torch/cq/code_QA_data/train/train.answer')


In [42]:
import random

def get_length_distribution(answers):
    lengths = [len(answer.split()) for answer in answers]  # Word count
    distribution = {length: lengths.count(length) for length in set(lengths)}
    return distribution

def truncate_comments(comments, distribution):
    results = []
    for comment in comments:
        # Select a length based on the distribution
        length = random.choices(list(distribution.keys()), weights=distribution.values(), k=1)[0]
        words = comment.split()
        
        # Truncate the string
                # Calculate the index for truncation
        if len(words) > length:  # Ensure there are enough words
            truncated_string = ' '.join(words[:-length])  # All words except the last 'length' words
            truncated_part = ' '.join(words[-length:])  # The last 'length' words
        else:
            truncated_string = comment.strip()  # If not enough words, take the whole comment
            truncated_part = ""  # No part to truncate
        
        results.append((truncated_string.strip(), truncated_part.strip()))
    
    return results


# Get the distribution from answers
distribution = get_length_distribution(answers)

# Truncate comments based on that distribution
truncated_results = truncate_comments(comments, distribution)

truncated_strings = []
truncated_parts = []

for truncated_string, truncated_part in truncated_results:
    truncated_strings.append(truncated_string)
    truncated_parts.append(truncated_part)


In [43]:
model_id = "Salesforce/codet5p-770m"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

In [ ]:
code_comment=[]
for code,comment in zip(codes,truncated_strings):
    temp=fit_in_shape(code,comment,tokenizer)
    code=temp[0]
    comment=temp[1]
    code_comment.append((code+tokenizer.sep_token+comment))


In [50]:
# Write truncated strings to a file
with open('/home/arya/my_python/my_torch/cq/par_des_gen/code_comment.data', 'w') as f:
    for string in code_comment:
        f.write(f"{string}\n")

# Write truncated parts to a file
with open('/home/arya/my_python/my_torch/cq/par_des_gen/truncated_part.data', 'w') as f:
    for part in truncated_parts:
        f.write(f"{part}\n")


In [51]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('/home/arya/my_python/my_torch/cq/')

In [52]:
# Define code snippet and question
import torch
code_snippet = """
public Map Entry Info Key ( Object value , Object key , Reference Queue < Object > queue ) { super ( value , queue ) ; hash Code = hash Code ^ key . hash Code ( ) ; key = key ; }
"""
question = "What does the code create ?"

# Prepare the input
input_text = f"code: {code_snippet}"+tokenizer.sep_token+f"question: {question}<extra_id_0>"
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate the model's response
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=150, num_beams=10, early_stopping=True)

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Model's response:", response)

Model's response: a new map entry
